# Lennard Jones Surface Energy

> **2024 Molecular Foundry Annual User Meeting**  
> _Digital Science Communication: Reproducibility, Reactivity, and Web Interfaces_  
> Georgios Varnavides | 08/12/2024

In [13]:
import sys
if "pyodide" in sys.modules:
  import micropip
  await micropip.install('ipywidgets')
  await micropip.install('ipympl')

%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

from IPython.display import display
import ipywidgets

dpi = 72

In [2]:
xx, yy = np.meshgrid(
    np.arange(-7.5,8.5,1.0),
    np.arange(-7.5,8.5,1.0),
    indexing='ij'
)

compressive_strain = 0.975
pts_initial = compressive_strain * np.stack((xx.ravel(),yy.ravel()),-1)
pts_initial.shape

(256, 2)

In [3]:
def distance_matrix(pts):
    """ efficiently computes the distance matrix between a set of points """
    # cast as complex array
    pts_complex = pts.view(dtype=np.complex128)
    dists = np.abs(pts_complex.T - pts_complex)
    return dists

def lennard_jones_energies(pts):
    """ efficiently compute array of pairwise LJ energies, summing over rows """
    n = pts.shape[0]
    dists = distance_matrix(pts) + np.identity(n)
    dists_sixth = dists**6
    return np.sum(1/dists_sixth**2 - 2/dists_sixth,0)/2 + 0.5

In [4]:
def slice_particle_half(pts, angle_deg):
    """
    Slices particle in half according to angle_deg,
    returning the displaced positions and computed energies
    """
    # slice
    angle = np.deg2rad(angle_deg)
    left_of_slice = (np.tan(angle % np.pi) * (pts[:,0]+1e-6)) < pts[:,1]
    pts_left = pts[left_of_slice]
    pts_right = pts[~left_of_slice]

    # energies
    energies_left = lennard_jones_energies(pts_left)
    energies_right = lennard_jones_energies(pts_right)

    # displace
    pts_left[:,0] -= np.sin(angle)
    pts_left[:,1] += np.cos(angle)
    pts_right[:,0] += np.sin(angle)
    pts_right[:,1] -= np.cos(angle)

    # populate
    new_pts = np.empty_like(pts)
    new_pts[left_of_slice] = pts_left
    new_pts[~left_of_slice] = pts_right

    energies = np.empty(pts.shape[0],dtype=energies_left.dtype)
    energies[left_of_slice] = energies_left
    energies[~left_of_slice] = energies_right
    
    return new_pts, energies

In [14]:
angle_deg = 45
angle = np.deg2rad(angle_deg)
pts, energies = slice_particle_half(pts_initial,angle_deg)

with plt.ioff():
    fig, ax = plt.subplots(figsize=(6,6.5),dpi=dpi)

scat = ax.scatter(
    pts[:,0],
    pts[:,1],
    c=energies,
    s=325,
    cmap='turbo',
    vmin=-3,
    vmax=0,
)

line, = ax.plot(
    [12.5*np.cos(angle),-12.5*np.cos(angle)],
    [12.5*np.sin(angle),-12.5*np.sin(angle)],
    linestyle='--',
    color='k',
) 

ax_divider = make_axes_locatable(ax)
cax = ax_divider.append_axes("bottom", size="3%", pad=0)
cbar = fig.colorbar(scat, cax=cax, orientation="horizontal",)
cbar.set_label("normalized Lennard-Jones interatomic potential energy")

ax.set_aspect("equal")
ax.axis("off")
ax.set_xlim([-10,10])
ax.set_ylim([-10,10])

fig.tight_layout()

In [15]:
style = {
    'description_width': 'initial',
}

layout = ipywidgets.Layout(
    width=f"{4.5*dpi}px",
)

angle_slider = ipywidgets.FloatSlider(
    value = angle_deg, min = -89.5, max = 90, 
    step = 0.5,
    description = r"slice angle, $\theta$ [°]",
    style = style,
    layout = layout,
)

def update_scatter(angle_deg):
    """ """
    angle = np.deg2rad(angle_deg)
    pts, energies = slice_particle_half(pts_initial,angle_deg)
    scat.set_offsets(pts)
    scat.set_array(energies)
    line.set_data(
        [12.5*np.cos(angle),-12.5*np.cos(angle)],
        [12.5*np.sin(angle),-12.5*np.sin(angle)],
    )
    fig.canvas.draw_idle()
    return None

fig.canvas.resizable = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.toolbar_visible = False
fig.canvas.layout.width = f"{6*dpi}px"
fig.canvas.layout.height = f"{6.5*dpi}px"

ipywidgets.widgets.interactive_output(
    update_scatter, 
    {
        'angle_deg':angle_slider,
    },
);

In [16]:
#| label: app:surface_energy

display(
    ipywidgets.VBox(
        [
            angle_slider,
            fig.canvas,
        ]
    )
)